In [1]:
import pandas as pd
import os
import re
import xmltodict

In [2]:
os.getcwd()
os.path.exists('data\\reviewed_human_protein.xml')


True

In [3]:
# This takes a couple minutes
with open('data/reviewed_human_protein.xml') as f:
    human_protein_dict = xmltodict.parse(f.read())

In [4]:
human_protein_dict.keys()

odict_keys(['uniprot'])

In [5]:
human_protein_dict['uniprot'].keys()

odict_keys(['@xmlns', '@xmlns:xsi', '@xsi:schemaLocation', 'entry', 'copyright'])

In [14]:
entry = human_protein_dict['uniprot']['entry'][0]
entry.keys()

odict_keys(['@dataset', '@created', '@modified', '@version', 'accession', 'name', 'protein', 'gene', 'organism', 'reference', 'comment', 'dbReference', 'proteinExistence', 'keyword', 'feature', 'evidence', 'sequence'])

In [15]:
len(human_protein_dict['uniprot']['entry'])

20394

In [101]:
def entryCommentParser(entry_name, comments):
    comm_list = []
    for comment in comments:
        comm_arr = [entry_name]
        comm_type = comment.get("@type",None)
        value = None
        if comm_type == 'interaction':
            for interaction in comment.get('interactant'):
                value = interaction.get('@intactId')
        elif comm_type == 'similarity':
            None
        comm_arr.extend([comm_type, value])
        comm_list.append(comm_arr)
    return comm_list

def secStrucParser(entry_name, features):
    """Right now I only want features that are annotations of;
    
    -strand (beta sheet)
    -helix  (alpha helix)
    -turn   (highly structured secondary structure)
    
    """
    
#     types = []
    feat_arr = []
    for feature in features:
        feat_type = feature.get("@type")
        if feat_type in ['strand','helix','turn']:
            feat_locations = feature.get('location')
            feat_begin = feat_locations.get('begin').get('@position')
            feat_end = feat_locations.get('end').get('@position')
            feat_arr.append([entry_name, feat_type, feat_begin, feat_end])
    return feat_arr
#     return types

def entryParser(entry):
    # Get name
    entry_name = entry.get('name')
    # Get sequence
    entry_seq = entry.get('sequence').get('#text').replace("\n","").strip()
    # Parse features
    entry_features = entry.get('feature')
    sec_struc_list = secStrucParser(entry_name, entry_features)
    # Parse comments
    #entry_comments = entry.get('comment', None)
    #if entry_comments != None:
    #    comm_list = entryCommentParser(entry_name, entry_comments)
    return entry_name, entry_seq, sec_struc_list #, comm_list #entry_gene

#for entry in human_protein_dict['uniprot']['entry']:
# 
entryParser(human_protein_dict['uniprot']['entry'][3])

('RAB10_HUMAN',
 'MAKKTYDLLFKLLLIGDSGVGKTCVLFRFSDDAFNTTFISTIGIDFKIKTVELQGKKIKLQIWDTAGQERFHTITTSYYRGAMGIMLVYDITNGKSFENISKWLRNIDEHANEDVERMLLGNKCDMDDKRVVPKGKGEQIAREHGIRFFETSAKANINIEKAFLTLAEDILRKTPVKEPNSENVDISSGGGVTGWKSKCC',
 [['RAB10_HUMAN', 'strand', '7', '15'],
  ['RAB10_HUMAN', 'helix', '22', '31'],
  ['RAB10_HUMAN', 'strand', '43', '53'],
  ['RAB10_HUMAN', 'strand', '56', '65'],
  ['RAB10_HUMAN', 'helix', '69', '71'],
  ['RAB10_HUMAN', 'helix', '72', '75'],
  ['RAB10_HUMAN', 'helix', '76', '78'],
  ['RAB10_HUMAN', 'turn', '79', '81'],
  ['RAB10_HUMAN', 'strand', '83', '90'],
  ['RAB10_HUMAN', 'helix', '94', '98'],
  ['RAB10_HUMAN', 'helix', '100', '110'],
  ['RAB10_HUMAN', 'strand', '116', '122'],
  ['RAB10_HUMAN', 'helix', '127', '129'],
  ['RAB10_HUMAN', 'helix', '134', '143'],
  ['RAB10_HUMAN', 'strand', '147', '150'],
  ['RAB10_HUMAN', 'turn', '153', '156'],
  ['RAB10_HUMAN', 'strand', '157', '159'],
  ['RAB10_HUMAN', 'helix', '160', '172']])

In [ ]:
protein_df = pd.DataFrame(columns=["protein","sequence"])
comment_df = pd.DataFrame(columns=["protein","comm_type","value"])
sec_struc_df = pd.DataFrame(columns=["protein","sec_struc_type","begin","end"])